In [1]:
import os
import sys
import pandas as pd
import json
import numpy as np
os.chdir(sys.path[0])
f=open(r"analysis_input.json")
input=json.load(f)
f.close()
df=pd.read_csv("Desired.csv", index_col=0, na_values='')

In [2]:
df

,Line 1,Line 2,Line 3,Line 4,Line 5,Line 6,desired
4,Damage when attacking boss monsters +30%,INT +9%,Ignore Enemy Defense +30%,ATT +9%,DEX +9%,DEX +9%,2
15,MATT +12%,ATT +9%,LUK +9%,ATT +9%,STR +9%,DEX +9%,2
30,STR +12%,STR +9%,Ignore Enemy Defense +30%,MATT +9%,5% chance to ignore 40% damage when hit,Ignore Enemy Defense +30%,2
37,+1 ATT per 10 character levels,Ignore Enemy Defense +30%,STR +9%,Damage when attacking boss monsters +20%,ATT +9%,STR +9%,2
38,INT +12%,ATT +9%,Damage when attacking boss monsters +20%,ATT +9%,Ignore Enemy Defense +30%,STR +9%,3
...,...,...,...,...,...,...,...
99954,ATT +12%,Critical Chance +9%,Damage when attacking boss monsters +30%,All Stats +6%,Critical Chance +9%,ATT +9%,2
99960,Ignore Enemy Defense +40%,ATT +9%,Damage +9%,STR +9%,All Stats +6%,LUK +9%,2
99965,10% chance to ignore 40% damage when hit,ATT +9%,Damage when attacking boss monsters +30%,STR +9%,Ignore Enemy Defense +30%,DEX +9%,2
99979,Damage when attacking boss monsters +40%,5% chance to ignore 40% damage when hit,ATT +9%,ATT +9%,STR +9%,MATT +9%,2


In [3]:

class keep:
    all=[]
    def __init__(self,name, before='', after=''):
        self.name=name
        self.before=before
        self.after=after
        self.df=pd.DataFrame({})
        keep.all.append(self)
    def init_df(self,df,column_header,number):
        self.df[self.name+' '+str(number)]=df[column_header].apply(lambda x:self.get_value(x))
        if len(df.keys())>3:
            self.summary=self.df.apply(keep.top3_edit, axis=1) ## maybe i use this only for hexa
        else:
            self.summary=self.df.sum(axis=1)
    def inside(self, string):
        if string.endswith(self.after) and string.startswith(self.before):
            return True
        else:
            return False
    def get_value(self, string):
        if self.inside(string):
            x=string.replace(self.before,"",1)
            x=x.removesuffix(self.after)
            return int(x)
        else: return 0 # maybe changing to  return none
    def top3(row):
        temp_list=[]
        for x in row[row.keys()].values.tolist():
            if not x==0:
                temp_list.append(x)
        if len(temp_list)>3:
            temp_list.sort(reverse=True)
            return sum(temp_list[0:3])
        else:
            return sum(temp_list)
    def top3_edit(row):
        temp_list=row[row.keys()].values.tolist()   
        temp_list= list(filter(lambda num:num>0,temp_list))
        if len(temp_list)>3:
            temp_list.sort(reverse=True)
            return sum(temp_list[0:3])
        else:
            return sum(temp_list)
    def inside_all(string):
        for x in keep.all:
            if x.inside(string):
                return True
        return False
before=0
after=1
keep_dict={}
for x in input.keys():
    keep_dict[x]=keep(x,input[x][before],input[x][after])
del before, after, input
#theres 2 collections keep_dict and keep.list


In [4]:
header=df.keys()
header=list(filter(lambda x: x.startswith("Line "),header))
header

['Line 1', 'Line 2', 'Line 3', 'Line 4', 'Line 5', 'Line 6']

In [5]:
count=0
for x in header:
    count+=1
    df[x]=df[x].apply(lambda x: x if keep.inside_all(x) else '')
    for y in keep.all:
        y.init_df(df,x,count)
for x in keep.all:
    df[x.name]=x.summary


In [6]:
df.to_csv("analysed.csv")

In [7]:
df

,Line 1,Line 2,Line 3,Line 4,Line 5,Line 6,desired,Attack,IED
4,,,Ignore Enemy Defense +30%,ATT +9%,,,2,9,30
15,,ATT +9%,,ATT +9%,,,2,18,0
30,,,Ignore Enemy Defense +30%,,,Ignore Enemy Defense +30%,2,0,60
37,,Ignore Enemy Defense +30%,,,ATT +9%,,2,9,30
38,,ATT +9%,,ATT +9%,Ignore Enemy Defense +30%,,3,18,30
...,...,...,...,...,...,...,...,...,...
99954,ATT +12%,,,,,ATT +9%,2,21,0
99960,Ignore Enemy Defense +40%,ATT +9%,,,,,2,9,40
99965,,ATT +9%,,,Ignore Enemy Defense +30%,,2,9,30
99979,,,ATT +9%,ATT +9%,,,2,18,0
